In [1]:
import numpy as np
import pandas as pd
import pprint
from scapy.all import *
from collections import defaultdict
from IPy import IP as IPy
#from manuf import manuf

In [2]:
# change this to the filename that you want to convert to a dataframe

filename = "./packet_capture.pcap"

In [3]:
packets = rdpcap(filename)

In [33]:
p_dict = {'Ether': ['dst', 'src', 'type'],
          'IP': ['version', 'ihl', 'tos', 'len', 'id', 'flags', 'frag', 'ttl', 'proto', 'chksum', 'src', 'dst'],
          'TCP': ['sport', 'dport', 'seq', 'ack', 'dataofs', 'reserved', 'flags', 'window', 'chksum', 'urgptr', 'options'],
          'Raw': ['load']       
         }


feature_list = []

# Probably a better way to do this other than 3 nested loops!
# On the small file takes 5 secs to run but on a bigger file will probalby be -a lot- worse
# Goes through each packet and each of the key value pairs in the dict above to grab
# each value from the raw file to make into a dataframe

count = 0
for p in packets:
    feature_dict = defaultdict(str)
    for key in p_dict.keys():
        for cat in p_dict[key]:
            col_key = key + '_' + cat
            call = '"%' + key + '.' + cat + '%"'
            #print(call)
            
            try:           
                feature_dict[col_key] = p.sprintf(call)
            except:
                feature_dict[col_key] = 'Null'
    
    feature_list.append(feature_dict)
    count += 1
    if count > 5:
        break

In [36]:
df = pd.DataFrame(feature_list)

In [37]:
df

,Ether_dst,Ether_src,Ether_type,IP_chksum,IP_dst,IP_flags,IP_frag,IP_id,IP_ihl,IP_len,...,TCP_chksum,TCP_dataofs,TCP_dport,TCP_flags,TCP_options,TCP_reserved,TCP_seq,TCP_sport,TCP_urgptr,TCP_window
0,"""e0:55:3d:02:41:21""","""84:38:35:5a:70:40""","""0x8100""","""0xacee""","""151.101.1.254""","""DF""","""0""","""12886""","""5""","""154""",...,"""0xea63""","""8""","""https""","""PA""","""[('NOP', None), ('NOP', None), ('Timestamp', ...","""0""","""3463304684""","""59554""","""0""","""9675"""
1,"""e0:55:3d:02:41:21""","""84:38:35:5a:70:40""","""0x8100""","""0x27c8""","""151.101.1.254""","""DF""","""0""","""45690""","""5""","""1436""",...,"""0x2c12""","""8""","""https""","""A""","""[('NOP', None), ('NOP', None), ('Timestamp', ...","""0""","""3463304786""","""59554""","""0""","""9675"""
2,"""e0:55:3d:02:41:21""","""84:38:35:5a:70:40""","""0x8100""","""0x56c""","""151.101.1.254""","""DF""","""0""","""55125""","""5""","""797""",...,"""0x213c""","""8""","""https""","""PA""","""[('NOP', None), ('NOP', None), ('Timestamp', ...","""0""","""3463306170""","""59554""","""0""","""9675"""
3,"""84:38:35:5a:70:40""","""e0:55:3d:02:41:21""","""0x8100""","""0x5634""","""192.168.1.14""","""DF""","""0""","""36982""","""5""","""52""",...,"""0x777c""","""8""","""59554""","""A""","""[('NOP', None), ('NOP', None), ('Timestamp', ...","""0""","""740824593""","""https""","""0""","""338"""
4,"""84:38:35:5a:70:40""","""e0:55:3d:02:41:21""","""0x8100""","""0xb301""","""192.168.1.14""","""""","""0""","""28638""","""5""","""52""",...,"""0x58aa""","""8""","""59611""","""A""","""[('NOP', None), ('NOP', None), ('Timestamp', ...","""0""","""396759928""","""https""","""0""","""275"""
5,"""e0:55:3d:02:41:21""","""84:38:35:5a:70:40""","""0x8100""","""0x115c""","""216.58.195.243""","""DF""","""0""","""52053""","""5""","""98""",...,"""0xa45e""","""8""","""https""","""PA""","""[('NOP', None), ('NOP', None), ('Timestamp', ...","""0""","""3235869591""","""59611""","""0""","""4096"""


In [24]:
list(df)

['Ethernet_dst',
 'Ethernet_src',
 'Ethernet_type',
 'IP_chksum',
 'IP_dst',
 'IP_flags',
 'IP_frag',
 'IP_id',
 'IP_ihl',
 'IP_len',
 'IP_proto',
 'IP_src',
 'IP_tos',
 'IP_ttl',
 'IP_version',
 'Raw_load',
 'TCP_ack',
 'TCP_chksum',
 'TCP_dataofs',
 'TCP_dport',
 'TCP_flags',
 'TCP_options',
 'TCP_reserved',
 'TCP_seq',
 'TCP_sport',
 'TCP_urgptr',
 'TCP_window']